# (CONSIGNA) 3er entrega - Sistemas de recomendación


En esta entrega vamos a trabajar con un sistema de recomendación.

Vamos a aprovechar la competencia de Telecom que vimos ya que los datos son reales y están buenos para practicar.

En el siguiente repositorio pueden encontrar el significado de cada columna de los datasets:
https://github.com/Datathon2021/Recomendador

Consigna:

- Dividir set en train y test. Tomar como train los datos hasta el 1 de marzo de 2021. Desde el 1ro de marzo en adelante, reservar para test.
- Desarrollar un recomendador. El recomendador debe ser capaz de generar recomendaciones para TODOS los usuarios (incluyendo los cold start que no tengan visualizaciones en el set de train). Generar 20 recomendaciones por usuario.
- Las recomendaciones tienen que ser para cada account_id y hay que recomendar content_id (NO asset_id). Pueden encontrar esto en el repositorio de la competencia.
- Los contenidos que recomienden, no tienen que haber sido vistos previamente por los usuarios (filtrar).
- Evaluarlo con MAP.

Recomendaciones:
- En este caso no tenemos ratings explícitos como los casos que vimos, deben generar ustedes estos ratings mediante algún criterio. Lo más simple podría ser utilizar ratings binarios (lo vió / no lo vió).
- Hay una columna que nos indica hasta cuando va a estar disponible el contenido
- La columna **end_vod_date**: "fecha de finalización de la disponibilidad del activo en la plataforma" puede llegar a serles muy útil. ¿Tiene sentido recomendar algo que no va a estar disponible en el set de test? (a partir del 1 de marzo de 2021).
- Comiencen con algo SIMPLE. No se compliquen con todas las columnas que tiene el dataset. No van a necesitar usar todas, muchas columnas podrán descartarlas dependiendo del enfoque que tomen.

Datos:
- Se encuentran adjuntos en la entrega

Fecha de entrega: 21/08/2024.

Pueden subirlo a un repositorio de github (público) y subir el link. De paso les sirve para ir armando su perfil de github con algunos proyectos 😉.


# DESARROLLO

In [6]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.sparse import csr_matrix

## PREPARACIÓN DE DATOS

In [7]:
"""
train.csv
Este dataset contiene los registros de visualizaciones de contenidos de Flow del formato video on demand (VOD), correspondiente a una muestra aleatoria de más de 113 mil perfiles. A continuación, se detalla el diccionario de variables de esta tabla:

customer_id: código de identificación de cada cliente de Flow (puede tener asociados uno o más account_id)
account_id: código de identificación de cada perfil de Flow (se corresponde con un único customer_id)
device_type: indica el tipo de dispositivo desde el que se efectuó la visualización. Las categorías posibles son:
CLOUD: cliente web
PHONE: teléfono celular
STATIONARY: smart TV
STB: set-top box, el decodificador Flow
TABLET
asset_id: código de identificación de cada activo (video) disponible en la plataforma
tunein: fecha y hora de inicio de cada visualización
tuneout: fecha y hora de finalización de cada visualización
resume: variable dummy que indica si se reanuda un consumo anterior del mismo asset_id
"""

# Apertura del dataset
train_df = pd.read_csv('train.csv')
train_df.head()

,customer_id,account_id,device_type,asset_id,tunein,tuneout,resume
0,0,90627,STATIONARY,18332.0,2021-02-18 22:52:00.0,2021-02-18 23:35:00.0,0
1,0,90627,STATIONARY,24727.0,2021-03-24 23:17:00.0,2021-03-25 00:01:00.0,0
2,1,3387,STB,895.0,2021-03-15 10:05:00.0,2021-03-15 10:23:00.0,0
3,1,3387,STB,895.0,2021-03-15 10:23:00.0,2021-03-15 11:18:00.0,1
4,1,3387,STB,26062.0,2021-03-16 09:24:00.0,2021-03-16 09:44:00.0,0


In [8]:
# Mantener solo las columnas relevantes
train_df = train_df[['account_id', "asset_id", "tunein"]]
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3657801 entries, 0 to 3657800
Data columns (total 3 columns):
 #   Column      Dtype  
---  ------      -----  
 0   account_id  int64  
 1   asset_id    float64
 2   tunein      object 
dtypes: float64(1), int64(1), object(1)
memory usage: 83.7+ MB


In [9]:
# Agregar una columna 'watched' con valor 1 para todas las filas
train_df['watched'] = 1

train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3657801 entries, 0 to 3657800
Data columns (total 4 columns):
 #   Column      Dtype  
---  ------      -----  
 0   account_id  int64  
 1   asset_id    float64
 2   tunein      object 
 3   watched     int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 111.6+ MB


In [10]:
"""
metadata.csv
Contiene la metadata asociada a cada uno de los contenidos. Las variables incluidas son:

asset_id: código de identificación de cada activo (video) disponible en Flow
content_id: código de identificación que agrupa los distintos asset_id asociados a un mismo contenido (por ejemplo, cada episodio de una misma serie tiene su propio asset_id, mientras que la serie se identifica con un content_id único)
title: título
reduced_title: título reducido
episode_title: título del episodio (válido para contenidos episódicos, como las series)
show_type: tipo de show - las categorías son autorreferenciales con excepción de “Rolling”, que indica que se trata de una serie incompleta, y “Web”, la cual remite a contenidos pensados íntegramente en formato digital (series web) -
released_year: año de lanzamiento
country_of_origin: país de origen – expresado con el código de dos letras propio del estándar internacional de normalización ISO 3166 -
category: categoría o género al que pertenece el contenido - puede haber una o más -
keywords: palabras clave o tags asociadas al contenido - puede haber una o más -
description: descripción (sinopsis)
reduced_desc: descripción (sinopsis) reducida
cast_first_name: nombre y apellido de los actores y actrices principales
credits_first_name: nombre y apellido del director o directora
run_time_min: duración total, expresada en minutos
audience: audiencia target
made_for_tv: variable dummy que indica si el contenido fue hecho para TV
close_caption: variable dummy que indica si el contenido posee subtítulos
sex_rating: variable dummy que indica si el contenido tiene escenas de sexo explícitas
violence_rating: variable dummy que indica si el contenido tiene escenas de violencia explícitas
language_rating: variable dummy que indica si el contenido posee lenguaje que puede ser considerado ofensivo o inapropiado
dialog_rating: variable dummy que indica si el contenido posee diálogos que pueden ser considerado ofensivos o inapropiados
fv_rating: variable dummy que indica si el contenido tiene rating de FV, que corresponde a público infantil con violencia ficticia
pay_per_view: variable dummy que indica si se trata de un alquiler
pack_premium_1: variable dummy que indica si se trata de un contenido exclusivo del pack premium 1
pack_premium_2: variable dummy que indica si se trata de un contenido exclusivo del pack premium 2
create_date: fecha de creación del activo
modify_date: fecha de modificación del activo
start_vod_date: fecha desde la cual el activo se encuentra disponible en la plataforma
end_vod_date: fecha de finalización de la disponibilidad del activo en la plataforma
"""

# Apertura del dataset
metadata_df = pd.read_csv('metadata.csv', sep=';', quotechar='"')
metadata_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33144 entries, 0 to 33143
Data columns (total 30 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   asset_id            33144 non-null  int64  
 1   content_id          33123 non-null  float64
 2   title               33144 non-null  object 
 3   reduced_title       33144 non-null  object 
 4   episode_title       28997 non-null  object 
 5   show_type           33140 non-null  object 
 6   released_year       33144 non-null  float64
 7   country_of_origin   33140 non-null  object 
 8   category            33144 non-null  object 
 9   keywords            33142 non-null  object 
 10  description         33142 non-null  object 
 11  reduced_desc        33144 non-null  object 
 12  cast_first_name     24412 non-null  object 
 13  credits_first_name  20590 non-null  object 
 14  run_time_min        33144 non-null  float64
 15  audience            33143 non-null  object 
 16  made

In [11]:
# Usar las columnas relevantes
metadata_df_aux = metadata_df.copy()
metadata_df_aux = metadata_df_aux[["asset_id", "content_id", "start_vod_date", "end_vod_date"]]
metadata_df_aux.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33144 entries, 0 to 33143
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   asset_id        33144 non-null  int64  
 1   content_id      33123 non-null  float64
 2   start_vod_date  33144 non-null  object 
 3   end_vod_date    33144 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 1.0+ MB


In [12]:
# Realizar el merge basado en 'asset_id'
merged_df = pd.merge(train_df, metadata_df_aux, on='asset_id', how='left')

# Verificar el resultad
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3657801 entries, 0 to 3657800
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   account_id      int64  
 1   asset_id        float64
 2   tunein          object 
 3   watched         int64  
 4   content_id      float64
 5   start_vod_date  object 
 6   end_vod_date    object 
dtypes: float64(2), int64(2), object(3)
memory usage: 195.3+ MB


In [13]:
merged_df.head()

,account_id,asset_id,tunein,watched,content_id,start_vod_date,end_vod_date
0,90627,18332.0,2021-02-18 22:52:00.0,1,2040.0,2021-02-18T00:00:00.0Z,2021-06-30T23:59:59.0Z
1,90627,24727.0,2021-03-24 23:17:00.0,1,2040.0,2021-03-18T00:00:00.0Z,2021-06-30T23:59:59.0Z
2,3387,895.0,2021-03-15 10:05:00.0,1,1983.0,2021-03-08T00:00:00.0Z,2021-03-28T23:59:00.0Z
3,3387,895.0,2021-03-15 10:23:00.0,1,1983.0,2021-03-08T00:00:00.0Z,2021-03-28T23:59:00.0Z
4,3387,26062.0,2021-03-16 09:24:00.0,1,729.0,2021-03-08T00:00:00.0Z,2021-03-28T23:59:00.0Z


In [14]:
# Borramos de asset_id, ya que no la vamos a utilizar.
columns_to_keep = ['account_id', 'content_id', 'watched', 'tunein', 'start_vod_date', 'end_vod_date']
watched_content = merged_df.copy()
watched_content = watched_content[columns_to_keep]

# Verificar el resultado
watched_content.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3657801 entries, 0 to 3657800
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   account_id      int64  
 1   content_id      float64
 2   watched         int64  
 3   tunein          object 
 4   start_vod_date  object 
 5   end_vod_date    object 
dtypes: float64(1), int64(2), object(3)
memory usage: 167.4+ MB


In [15]:
watched_content.head()

,account_id,content_id,watched,tunein,start_vod_date,end_vod_date
0,90627,2040.0,1,2021-02-18 22:52:00.0,2021-02-18T00:00:00.0Z,2021-06-30T23:59:59.0Z
1,90627,2040.0,1,2021-03-24 23:17:00.0,2021-03-18T00:00:00.0Z,2021-06-30T23:59:59.0Z
2,3387,1983.0,1,2021-03-15 10:05:00.0,2021-03-08T00:00:00.0Z,2021-03-28T23:59:00.0Z
3,3387,1983.0,1,2021-03-15 10:23:00.0,2021-03-08T00:00:00.0Z,2021-03-28T23:59:00.0Z
4,3387,729.0,1,2021-03-16 09:24:00.0,2021-03-08T00:00:00.0Z,2021-03-28T23:59:00.0Z


In [16]:
# Verificación valores nulos
nan_counts = watched_content.isna().sum()
nan_counts

account_id          0
content_id        142
watched             0
tunein              0
start_vod_date     22
end_vod_date       22
dtype: int64

In [17]:
# Eliminar filas con NaN en cualquier columna
watched_content = watched_content.dropna()

In [18]:
# Convertir las columnas a datetime
watched_content['tunein'] = pd.to_datetime(watched_content['tunein'])
watched_content['start_vod_date'] = pd.to_datetime(watched_content['start_vod_date'])
watched_content['end_vod_date'] = pd.to_datetime(watched_content['end_vod_date'])

# Verificar el resultado
watched_content.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3657659 entries, 0 to 3657800
Data columns (total 6 columns):
 #   Column          Dtype              
---  ------          -----              
 0   account_id      int64              
 1   content_id      float64            
 2   watched         int64              
 3   tunein          datetime64[ns]     
 4   start_vod_date  datetime64[ns, UTC]
 5   end_vod_date    datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](2), datetime64[ns](1), float64(1), int64(2)
memory usage: 195.3 MB


In [19]:
# Dar formato sin hora
watched_content['tunein'] = watched_content['tunein'].dt.strftime('%Y-%m-%d')
watched_content['start_vod_date'] = watched_content['start_vod_date'].dt.strftime('%Y-%m-%d')
watched_content['end_vod_date'] = watched_content['end_vod_date'].dt.strftime('%Y-%m-%d')

watched_content.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3657659 entries, 0 to 3657800
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   account_id      int64  
 1   content_id      float64
 2   watched         int64  
 3   tunein          object 
 4   start_vod_date  object 
 5   end_vod_date    object 
dtypes: float64(1), int64(2), object(3)
memory usage: 195.3+ MB


In [20]:
# Convertir las columnas a datetime
watched_content['tunein'] = pd.to_datetime(watched_content['tunein'])
watched_content['start_vod_date'] = pd.to_datetime(watched_content['start_vod_date'])
watched_content['end_vod_date'] = pd.to_datetime(watched_content['end_vod_date'])

watched_content.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3657659 entries, 0 to 3657800
Data columns (total 6 columns):
 #   Column          Dtype         
---  ------          -----         
 0   account_id      int64         
 1   content_id      float64       
 2   watched         int64         
 3   tunein          datetime64[ns]
 4   start_vod_date  datetime64[ns]
 5   end_vod_date    datetime64[ns]
dtypes: datetime64[ns](3), float64(1), int64(2)
memory usage: 195.3 MB


In [21]:
watched_content.head()

,account_id,content_id,watched,tunein,start_vod_date,end_vod_date
0,90627,2040.0,1,2021-02-18,2021-02-18,2021-06-30
1,90627,2040.0,1,2021-03-24,2021-03-18,2021-06-30
2,3387,1983.0,1,2021-03-15,2021-03-08,2021-03-28
3,3387,1983.0,1,2021-03-15,2021-03-08,2021-03-28
4,3387,729.0,1,2021-03-16,2021-03-08,2021-03-28


## DIVISIÓN DE DATOS

In [22]:
train_set = watched_content[(watched_content.tunein < datetime(year=2021, month=3, day=1))]
test_set = watched_content[(watched_content.tunein >= datetime(year=2021, month=3, day=1))]

## CREANDO LA MATRIZ DE INTERACCIONES

In [23]:
print("Train Shape:", train_set.shape)
print("Test Shape:", test_set.shape)

Train Shape: (2339040, 6)
Test Shape: (1318619, 6)


In [24]:
# Filtro los contenidos que no vas a estar disponibles para no recomendarlos
train_set = train_set[(train_set.end_vod_date >= datetime(year=2021, month=3, day=1))]
print("Train Shape:", train_set.shape)

Train Shape: (2062312, 6)


In [25]:
# Armo mi matriz de interaciones
matrix_columns = ["account_id", "content_id", "watched"]
interactions = train_set[matrix_columns]
interactions.head()

,account_id,content_id,watched
0,90627,2040.0,1
6,3388,2100.0,1
7,3388,2100.0,1
8,3388,2100.0,1
9,3388,2100.0,1


In [26]:
#Armo mi matrix pivot
interactions = interactions.drop_duplicates(subset=['account_id', 'content_id'])
interactions_matrix = interactions.pivot(index="account_id", columns="content_id", values="watched")

In [27]:
interactions_matrix.head()

content_id,0.0,1.0,3.0,6.0,7.0,8.0,9.0,11.0,12.0,13.0,...,4357.0,4358.0,4359.0,4360.0,4361.0,4362.0,4363.0,4364.0,4365.0,4366.0
account_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
interactions_matrix.shape

(97627, 3041)

In [29]:
# Convierto los NaN a 0
interactions_matrix = interactions_matrix.fillna(0)
interactions_matrix.head()

content_id,0.0,1.0,3.0,6.0,7.0,8.0,9.0,11.0,12.0,13.0,...,4357.0,4358.0,4359.0,4360.0,4361.0,4362.0,4363.0,4364.0,4365.0,4366.0
account_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
# Convierto a matriz esparsa
interactions_matrix_csr = csr_matrix(interactions_matrix.values)

In [31]:
interactions_matrix_csr

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 593052 stored elements and shape (97627, 3041)>

In [32]:
# Creo un dicionario de id de cuentas
account_ids = list(interactions_matrix.index)
accounts_dict = {}
counter = 0
for i in account_ids:
    accounts_dict[i] = counter
    counter += 1

In [33]:
accounts_dict

{0: 0,
 2: 1,
 3: 2,
 4: 3,
 5: 4,
 6: 5,
 7: 6,
 8: 7,
 10: 8,
 11: 9,
 12: 10,
 13: 11,
 14: 12,
 15: 13,
 16: 14,
 17: 15,
 18: 16,
 19: 17,
 20: 18,
 22: 19,
 23: 20,
 24: 21,
 25: 22,
 27: 23,
 28: 24,
 30: 25,
 31: 26,
 32: 27,
 33: 28,
 34: 29,
 35: 30,
 36: 31,
 37: 32,
 38: 33,
 39: 34,
 40: 35,
 41: 36,
 42: 37,
 44: 38,
 45: 39,
 46: 40,
 47: 41,
 49: 42,
 51: 43,
 52: 44,
 53: 45,
 54: 46,
 55: 47,
 56: 48,
 58: 49,
 59: 50,
 60: 51,
 61: 52,
 62: 53,
 63: 54,
 64: 55,
 65: 56,
 66: 57,
 67: 58,
 69: 59,
 70: 60,
 71: 61,
 72: 62,
 73: 63,
 74: 64,
 75: 65,
 76: 66,
 78: 67,
 79: 68,
 80: 69,
 81: 70,
 82: 71,
 83: 72,
 85: 73,
 87: 74,
 88: 75,
 89: 76,
 90: 77,
 91: 78,
 92: 79,
 93: 80,
 94: 81,
 95: 82,
 96: 83,
 97: 84,
 98: 85,
 101: 86,
 102: 87,
 103: 88,
 104: 89,
 105: 90,
 106: 91,
 107: 92,
 108: 93,
 109: 94,
 110: 95,
 111: 96,
 112: 97,
 113: 98,
 114: 99,
 115: 100,
 116: 101,
 117: 102,
 118: 103,
 119: 104,
 120: 105,
 121: 106,
 122: 107,
 123: 108,
 125:

## ENTRENAMIENTO DEL MODELO DE RECOMENDACIONES

In [34]:
pip install git+https://github.com/daviddavo/lightfm

  Cloning https://github.com/daviddavo/lightfm to c:\users\natalia\appdata\local\temp\pip-req-build-t_o3if_k
  Resolved https://github.com/daviddavo/lightfm to commit f0eb500ead54ab65eb8e1b3890337a7223a35114
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/daviddavo/lightfm 'C:\Users\Natalia\AppData\Local\Temp\pip-req-build-t_o3if_k'


In [35]:
from lightfm import LightFM

#Creamos el modelo con LightFM
model = LightFM(no_components=3, random_state=100, learning_rate=0.03)

c:\Users\Natalia\Desktop\Entrega 3\.venv\Lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [36]:
%%time

#Lo entrenamos con los datos de la matriz de interacción
model = model.fit(interactions_matrix_csr, epochs=10)

CPU times: total: 9.42 s
Wall time: 9.51 s


In [37]:
model

## EJEMPLO DE GENERACIÓN DE RECOMENDACIONES PARA UN USUARIO

In [38]:
# Genero las predicciones para un usuario especifico
account_x = account_ids[18]
n_accounts, n_contents = interactions_matrix.shape
content_ids = np.arange(n_contents)
preds = model.predict(user_ids=account_x, item_ids = content_ids)

In [39]:
preds

array([1.1731191, 0.9493715, 0.6109018, ..., 1.7433484, 3.145489 ,
       3.2268918], dtype=float32)

In [40]:
# Ordeno las prediciones segun su score, poniendo los con mayor score primero
scores = pd.Series(preds)
scores.index = interactions_matrix.columns
recomms_18 = list(pd.Series(scores.sort_values(ascending=False).index))[:20] #obtenemos las primeras 10

In [41]:
recomms_18

[2040.0,
 3806.0,
 3900.0,
 3598.0,
 2160.0,
 3381.0,
 3690.0,
 3550.0,
 3210.0,
 3384.0,
 2627.0,
 3775.0,
 116.0,
 3382.0,
 3711.0,
 3716.0,
 2942.0,
 3712.0,
 2827.0,
 1316.0]

In [43]:
# Me fijo cuales ya vio
train_set[(train_set.account_id==18)].content_id.unique()

array([2815., 3064., 3521., 2323., 3174., 2412., 1314., 3232.,  173.,
       3863., 1971.])

In [45]:
#EJEMPLO DE COLD START: El usuario todavia no vio nada
train_set.head()

,account_id,content_id,watched,tunein,start_vod_date,end_vod_date
0,90627,2040.0,1,2021-02-18,2021-02-18,2021-06-30
6,3388,2100.0,1,2021-01-01,2018-05-08,2021-06-30
7,3388,2100.0,1,2021-01-02,2018-05-08,2021-06-30
8,3388,2100.0,1,2021-01-02,2019-07-26,2021-06-30
9,3388,2100.0,1,2021-01-04,2019-07-26,2021-06-30


In [46]:
train_set.groupby("content_id", as_index=False).agg({"account_id":"nunique"})

,content_id,account_id
0,0.0,64
1,1.0,43
2,3.0,16
3,6.0,1210
4,7.0,17
...,...,...
3036,4362.0,612
3037,4363.0,132
3038,4364.0,157
3039,4365.0,932


In [47]:
# Obtener una lista con los 20 contenidos más populares
popularity_df = train_set.groupby("content_id", as_index=False).agg({"account_id":"nunique"}).sort_values(by="account_id", ascending=False)
popularity_df.columns=["content_id", "popularity"]
popularity_df.head()

,content_id,popularity
1083,2040.0,11481
2828,3806.0,8762
2922,3900.0,7960
2624,3598.0,3772
1201,2160.0,3506


In [48]:
popular_content = popularity_df.content_id.values[:20]
popularity_df.head(20).content_id.values
popular_content

array([2040., 3806., 3900., 3598., 2160., 3381., 3690., 3210., 3550.,
       3384., 2627., 3711., 3775., 3382.,  116., 3716., 1316., 2827.,
       3712.,  724.])

## GENERACIÓN DE RECOMENDACIONES PARA TODOS LOS USUARIOS

Ahora basándonos en el ejemplo anterior, vamos a generar 20 recomendaciones para todas las cuentas.

Debemos tener en cuenta:

Filtrar contenidos que la cuenta vio anteriormente
Si la cuenta no está en el set de train, recomendarle los 20 contenidos más populares

In [49]:
!pip install tqdm

In [50]:
from tqdm import tqdm

In [51]:
#definimos dict donde vamos a ir almacenando las recomendaciones
recomms_dict = {
    'account_id': [],
    'recomms': []
}

# Obtenemos cantidad de cuentas y cantidad de items
n_accounts, n_contents = interactions_matrix.shape
content_ids = np.arange(n_contents)

# Por cada cuenta del dataset de test, generamos recomendaciones
for account in tqdm(test_set.account_id.unique()):
    # Validar si el perfil se encuentra en la matriz de interacciones (interactions_matrix.index)
    if account in list(interactions_matrix.index):
        # Si el perfil esta en train, no es cold start. Usamos el modelo para recomendar
        account_x = accounts_dict[account] #buscamos el indice del perfil en la matriz (transformamos id a indice)

        # Generar las predicciones para la cuenta x
        preds = model.predict(user_ids=account_x, item_ids = content_ids)

        scores = pd.Series(preds)
        scores.index = interactions_matrix.columns
        scores = list(pd.Series(scores.sort_values(ascending=False).index))[:50]

        # Obtener listado de contenidos vistos anteriormente por el perfil (en el set de train)
        watched_contents = train_set[train_set.account_id == account].content_id.unique()

        # Filtrar contenidos ya vistos y quedarse con los primeros 20
        recomms = [x for x in scores if x not in watched_contents][:20]

        # Guardamos las recomendaciones en el diccionario
        recomms_dict['account_id'].append(account)
        recomms_dict['recomms'].append(scores)
    
    # En este else trataremos a los perfiles que no están en la matriz (cold start)
    else:
        recomms_dict['account_id'].append(account)
        # Les recomendamos contenido popular
        recomms_dict['recomms'].append(popular_content)


100%|██████████| 87624/87624 [19:57<00:00, 73.17it/s] 


In [52]:
# Convertimos las recomendaciones a un dataframe
recomms_df = pd.DataFrame(recomms_dict)
recomms_df

,account_id,recomms
0,90627,"[2040.0, 3806.0, 3900.0, 3598.0, 2160.0, 3381...."
1,3387,"[2040.0, 3806.0, 3900.0, 3598.0, 2160.0, 3381...."
2,3388,"[2040.0, 3806.0, 3900.0, 3598.0, 2160.0, 3381...."
3,3389,"[2040.0, 3806.0, 3900.0, 3598.0, 2160.0, 3381...."
4,3393,"[2040.0, 3806.0, 3900.0, 3598.0, 3381.0, 2160...."
...,...,...
87619,3382,"[2040.0, 3806.0, 3900.0, 3598.0, 2160.0, 3381...."
87620,3383,"[2040.0, 3806.0, 3900.0, 3598.0, 2160.0, 3690...."
87621,3385,"[2040.0, 3806.0, 3900.0, 3598.0, 2160.0, 3690...."
87622,82934,"[2040.0, 3806.0, 3900.0, 3598.0, 3381.0, 2160...."


In [53]:
# Guardar el DataFrame en un archivo CSV
recomms_df.to_csv('recomms.csv', index=False)

## CALCULO DE MÉTRICAS (MAP)

In [54]:
test_set.head()

,account_id,content_id,watched,tunein,start_vod_date,end_vod_date
1,90627,2040.0,1,2021-03-24,2021-03-18,2021-06-30
2,3387,1983.0,1,2021-03-15,2021-03-08,2021-03-28
3,3387,1983.0,1,2021-03-15,2021-03-08,2021-03-28
4,3387,729.0,1,2021-03-16,2021-03-08,2021-03-28
5,3387,729.0,1,2021-03-16,2021-03-08,2021-03-28


In [55]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1318619 entries, 1 to 3657800
Data columns (total 6 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   account_id      1318619 non-null  int64         
 1   content_id      1318619 non-null  float64       
 2   watched         1318619 non-null  int64         
 3   tunein          1318619 non-null  datetime64[ns]
 4   start_vod_date  1318619 non-null  datetime64[ns]
 5   end_vod_date    1318619 non-null  datetime64[ns]
dtypes: datetime64[ns](3), float64(1), int64(2)
memory usage: 70.4 MB


In [56]:
# Agrupo todos los contenidos que vio cada cuenta
ideal_recomms = test_set[test_set.watched==1]\
                  .groupby(["account_id"], as_index=False)\
                  .agg({"content_id": "unique"})
ideal_recomms.head()

,account_id,content_id
0,2,[433.0]
1,3,"[1949.0, 2409.0, 4010.0, 3169.0, 3487.0, 3980...."
2,4,"[2314.0, 728.0, 4129.0, 2344.0, 2341.0, 513.0,..."
3,5,[2259.0]
4,6,"[3902.0, 3211.0, 2900.0, 3386.0, 4065.0, 3388.0]"


In [57]:
ideal_recomms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87624 entries, 0 to 87623
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   account_id  87624 non-null  int64 
 1   content_id  87624 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.3+ MB


In [58]:
# Vamos a unir en un mismo dataframe las recomendaciones nuestras con el set ideal.
df_map = ideal_recomms.merge(recomms_df, how="left", left_on="account_id", right_on="account_id")[["account_id", "content_id", "recomms"]]
df_map.columns = ["account_id", "ideal", "recomms"]
df_map.head()

,account_id,ideal,recomms
0,2,[433.0],"[2040.0, 3806.0, 3900.0, 3598.0, 2160.0, 3381...."
1,3,"[1949.0, 2409.0, 4010.0, 3169.0, 3487.0, 3980....","[2040.0, 3806.0, 3900.0, 3598.0, 2160.0, 3690...."
2,4,"[2314.0, 728.0, 4129.0, 2344.0, 2341.0, 513.0,...","[2040.0, 3806.0, 3900.0, 3598.0, 2160.0, 3381...."
3,5,[2259.0],"[2040.0, 3806.0, 3900.0, 3598.0, 3381.0, 2160...."
4,6,"[3902.0, 3211.0, 2900.0, 3386.0, 4065.0, 3388.0]","[2040.0, 3806.0, 3900.0, 3598.0, 2160.0, 3381...."


In [59]:
df_map.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87624 entries, 0 to 87623
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   account_id  87624 non-null  int64 
 1   ideal       87624 non-null  object
 2   recomms     87624 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.0+ MB


In [60]:
aps = [] # Lista para almacenar la AP de cada usuario

for index, row in df_map.iterrows():
    pred = row['recomms']
    label = row['ideal']
    
    n = len(pred) # Cantidad de elementos recomendados
    
    arange = np.arange(n, dtype=np.int32) + 1. # Indexamos en base 1
    rel_k = np.isin(pred[:n], label) # Lista de booleanos que indican la relevancia de cada ítem
    
    if np.any(rel_k): # Solo procede si hay ítems relevantes
        tp = np.ones(rel_k.sum(), dtype=np.int32).cumsum() # Contador de verdaderos positivos
        
        denom = arange[rel_k] # Posiciones de los ítems relevantes
        
        # Calcula la precisión promedio para el usuario
        ap = (tp / denom).sum() / len(label)
    else:
        ap = 0.0

    aps.append(ap)

# Calcular el MAP
MAP = np.mean(aps)
print(f'Mean Average Precision = {round(MAP, 3)}')


Mean Average Precision = 0.065
